## 1. Look up the Adam optimization functions in PyTorch https://pytorch.org/docs/stable/optim.html . How does it work? Try at least one other optimization function with the diabetes dataset shown in class. How does the model perform with the new optimizer? Did it perform better or worse than Adam? Why do you think that is?

In [23]:
import pandas as pd
import torch

diabetes_df = pd.read_csv("diabetes.csv")
diabetes_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [24]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = diabetes_df.drop('Outcome', axis=1).values
y = diabetes_df['Outcome'].values

# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42, stratify=y)

# #Standardize
sc= StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

In [25]:
import torch.nn as nn
import torch.nn.functional as F #this has activation functions

# Creating tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

print(X_train)

tensor([[-0.8514, -0.9801, -0.4048,  ..., -0.6077,  0.3108, -0.7922],
        [ 0.3566,  0.1614,  0.4654,  ..., -0.3021, -0.1164,  0.5610],
        [-0.5494, -0.5045, -0.6223,  ...,  0.3726, -0.7649, -0.7076],
        ...,
        [-0.8514, -0.7582,  0.0303,  ...,  0.7800, -0.7861, -0.2847],
        [ 1.8665, -0.3142,  0.0303,  ..., -0.5695, -1.0194,  0.5610],
        [ 0.0546,  0.7322, -0.6223,  ..., -0.3149, -0.5770,  0.3073]])


In [26]:
class ANN_Model(nn.Module):
    def __init__(self, input_features=8, hidden1=20, hidden2=20, out_features =2):
        super().__init__()
        self.layer_1_connection = nn.Linear(input_features, hidden1)
        self.layer_2_connection = nn.Linear(hidden1, hidden2)
        self.out = nn.Linear(hidden2, out_features)
    
    def forward(self, x):
        #apply activation functions
        x = F.relu(self.layer_1_connection(x))
        x = F.relu(self.layer_2_connection(x))
        x = self.out(x)
        return x

In [27]:
torch.manual_seed(42)

#instantiate the model
model = ANN_Model()

In [84]:
# loss function
loss_function = nn.CrossEntropyLoss()

#optimizer
#optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [29]:
#run model through multiple epochs/iterations
final_loss = []
n_epochs = 500
for epoch in range(n_epochs):
    y_pred = model.forward(X_train)
    loss = loss_function(y_pred, y_train)
    final_loss.append(loss)
    
    if epoch % 10 == 1:
        print(f'Epoch number: {epoch} with loss: {loss.item()}')
    
    optimizer.zero_grad() #zero the gradient before running backwards propagation
    loss.backward() #for backward propagation 
    optimizer.step() #performs one optimization step each epoch

Epoch number: 1 with loss: 0.647413969039917
Epoch number: 11 with loss: 0.5237652659416199
Epoch number: 21 with loss: 0.45212870836257935
Epoch number: 31 with loss: 0.416485071182251
Epoch number: 41 with loss: 0.39122629165649414
Epoch number: 51 with loss: 0.36800774931907654
Epoch number: 61 with loss: 0.3417806625366211
Epoch number: 71 with loss: 0.3113643229007721
Epoch number: 81 with loss: 0.2804130017757416
Epoch number: 91 with loss: 0.24845130741596222
Epoch number: 101 with loss: 0.22124838829040527
Epoch number: 111 with loss: 0.19681058824062347
Epoch number: 121 with loss: 0.17579977214336395
Epoch number: 131 with loss: 0.15588246285915375
Epoch number: 141 with loss: 0.13907697796821594
Epoch number: 151 with loss: 0.12018003314733505
Epoch number: 161 with loss: 0.10505197942256927
Epoch number: 171 with loss: 0.09225479513406754
Epoch number: 181 with loss: 0.07996862381696701
Epoch number: 191 with loss: 0.0701649934053421
Epoch number: 201 with loss: 0.061516039

In [30]:
#predictions
y_pred = []

with torch.no_grad():
    for i, data in enumerate(X_test):
        prediction = model(data)
        y_pred.append(prediction.argmax().item())

In [31]:
from sklearn.metrics import accuracy_score
a_score = accuracy_score(y_test, y_pred)
print(a_score)

0.6948051948051948


In [32]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.74      0.76       100
           1       0.56      0.61      0.58        54

    accuracy                           0.69       154
   macro avg       0.67      0.68      0.67       154
weighted avg       0.70      0.69      0.70       154



In [69]:
torch.manual_seed(42)

In [85]:
#trying a new optimizer
optimizer_new = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [86]:
#run model through multiple epochs/iterations
final_loss = []
n_epochs = 500
for epoch in range(n_epochs):
    y_pred = model.forward(X_train)
    loss = loss_function(y_pred, y_train)
    final_loss.append(loss)
    
    if epoch % 10 == 1:
        print(f'Epoch number: {epoch} with loss: {loss.item()}')
    
    optimizer_new.zero_grad() #zero the gradient before running backwards propagation
    loss.backward() #for backward propagation 
    optimizer_new.step() #performs one optimization step each epoch

Epoch number: 1 with loss: 0.0019075236050412059
Epoch number: 11 with loss: 0.0019065699307247996
Epoch number: 21 with loss: 0.0019047590903937817
Epoch number: 31 with loss: 0.0019027445232495666
Epoch number: 41 with loss: 0.00190059095621109
Epoch number: 51 with loss: 0.001898408168926835
Epoch number: 61 with loss: 0.0018962433096021414
Epoch number: 71 with loss: 0.0018941123271360993
Epoch number: 81 with loss: 0.001892012543976307
Epoch number: 91 with loss: 0.0018898970447480679
Epoch number: 101 with loss: 0.0018877907423302531
Epoch number: 111 with loss: 0.0018855440430343151
Epoch number: 121 with loss: 0.0018834267975762486
Epoch number: 131 with loss: 0.0018813427304849029
Epoch number: 141 with loss: 0.0018791434122249484
Epoch number: 151 with loss: 0.0018770667957141995
Epoch number: 161 with loss: 0.00187488307710737
Epoch number: 171 with loss: 0.0018727665301412344
Epoch number: 181 with loss: 0.001870694919489324
Epoch number: 191 with loss: 0.001868623425252735

In [87]:
y_pred = []

with torch.no_grad():
    for i, data in enumerate(X_test):
        prediction = model(data)
        y_pred.append(prediction.argmax().item())

In [88]:
from sklearn.metrics import accuracy_score
a_score = accuracy_score(y_test, y_pred)
print(a_score)

0.6948051948051948


## 2. Write a function that lists and counts the number of divisors for an input value.

In [69]:
def divisor(n):
    for i in range(n):     
        x = [i for i in range(1,n+1) if not n % i]
    return x

my_list = divisor(15)

my_list 

[1, 3, 5, 15]

In [70]:
len(my_list)

4

In [71]:
#converting to string
list_string = map(str, my_list) 
new_list = list(list_string)

In [72]:
#def format(l):
    #return ",".join(new_list)
#format(new_list)

In [74]:
def format1(l):
    for i in l[:-1]:
        blank = ",".join(l[:-1]) + " and " + l[-1]
        return blank

num = str(len(new_list))
print("There are" +" "+ num  +" "+ "divisors:" + format1(new_list))   

There are 4 divisors:1,3,5 and 15
